In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [2]:
DATADIR = Path("data")

In [3]:
from data_processing import *
import data_processing as dp

from train import *

In [4]:
partitions = []
partition_list = [
    0
    if i in range(0, 4)
    else 1
    if i in range(5, 14 + 1)
    else 2
    if i in range(15, 27 + 1)
    else 0
    for i in range(0, 27 + 1)
]
partitions.append({i: v for i, v in enumerate(partition_list)})

partition_list = [
    0
    if i == 0
    else 1
    if i in range(1, 9 + 1)
    else 2
    if i in range(10, 19 + 1)
    else 3
    if i in range(20, 27 + 1)
    else 0
    for i in range(0, 27 + 1)
]
partitions.append({i: v for i, v in enumerate(partition_list)})


partition_list = [
    0
    if i in range(0, 4)
    else 1
    if i in range(5, 9 + 1)
    else 2
    if i in range(10, 14 + 1)
    else 3
    if i in range(15, 27 + 1)
    else 0
    for i in range(0, 27 + 1)
]
partitions.append({i: v for i, v in enumerate(partition_list)})

partitions.append(None)


partition_list = [
    1
    if i in range(1, 4 + 1)
    else 2
    if i in range(5, 9 + 1)
    else 3
    if i in range(10, 14 + 1)
    else 4
    if i in range(15, 19 + 1)
    else 5
    if i in range(20, 27 + 1)
    else 0
    for i in range(0, 27 + 1)
]
partitions.append({i: v for i, v in enumerate(partition_list)})

# Trying dailies additively

In [5]:
def all_combinations_scores(dailies_names = []):
    train_scores = []
    test_scores = []
    train_scores_sel = []
    test_scores_sel = []
    splits_by_participant = []
    test_take_first = []

    # No split by participant
    SPLIT_BY_PARTICIPANT = False
    for partition in partitions:
        res = train(
            TYPE="classification",
            TARGET="value",
            SPLIT_BY_PARTICIPANT=SPLIT_BY_PARTICIPANT,
            TEST_TAKE_FIRST=-1,
            SEED=550,
            return_csv=False,
            feature_selection=True,
            partition=partition,
            verbose=False,
            dailies_names=dailies_names
        )
        train_scores.append(res[0])
        test_scores.append(res[1])
        train_scores_sel.append(res[2])
        test_scores_sel.append(res[3])
        splits_by_participant.append(SPLIT_BY_PARTICIPANT)
        test_take_first.append(0)

    # Split by participant, take first 0,1,2,3,4
    SPLIT_BY_PARTICIPANT = True
    for TEST_TAKE_FIRST in range(0, 4 + 1):
        for partition in partitions:
            res = train(
                TYPE="classification",
                TARGET="value",
                SPLIT_BY_PARTICIPANT=SPLIT_BY_PARTICIPANT,
                TEST_TAKE_FIRST=TEST_TAKE_FIRST,
                SEED=550,
                return_csv=False,
                feature_selection=True,
                partition=partition,
                verbose=False,
                dailies_names=dailies_names
            )
            train_scores.append(res[0])
            test_scores.append(res[1])
            train_scores_sel.append(res[2])
            test_scores_sel.append(res[3])
            splits_by_participant.append(SPLIT_BY_PARTICIPANT)
            test_take_first.append(TEST_TAKE_FIRST)

    df = pd.DataFrame(
        {
            "partition": [
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
            ],
            "train": train_scores,
            "test": test_scores,
            "train (fs)": train_scores_sel,
            "test (fs)": test_scores_sel,
            "SPLIT": splits_by_participant,
            "TTF": test_take_first,
        }
    )
    dailies_substr = "_".join([str(x) for x in dailies_names])
    df.to_csv("partition_results_" + dailies_substr + ".csv", index=False)

    return df

## locations (base)

In [6]:
df = pd.read_csv("partition_results_.csv")
df

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",96.855984,70.857143,96.653144,71.428571,False,0
1,"0, 1-9, 10-19, 20-27",96.450304,69.714286,96.146045,66.285714,False,0
2,"0-4, 5-9, 10-14, 15-27",95.740365,57.142857,95.131846,59.428571,False,0
3,base,94.523327,50.285714,94.523327,53.714286,False,0
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",95.233266,45.714286,94.523327,45.714286,False,0
5,"0-4, 5-14, 15-27",96.800826,56.250000,96.594427,60.416667,True,0
6,"0, 1-9, 10-19, 20-27",96.284830,46.875000,95.975232,45.833333,True,0
7,"0-4, 5-9, 10-14, 15-27",94.943240,39.583333,94.530444,39.062500,True,0
8,base,94.117647,26.562500,93.911249,28.645833,True,0
9,"0, 1-4, 5-9, 10-14, 15-19, 20-27",93.808050,27.083333,93.292054,32.291667,True,0


In [7]:
df[df["train"] == df["train"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
25,"0-4, 5-14, 15-27",96.89781,61.538462,96.624088,64.615385,True,4


In [8]:
df[df["train (fs)"] == df["train (fs)"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",96.855984,70.857143,96.653144,71.428571,False,0


In [9]:
df[df["test"] == df["test"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
26,"0, 1-9, 10-19, 20-27",96.350365,76.923077,96.076642,73.846154,True,4


In [10]:
df[df["test (fs)"] == df["test (fs)"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
26,"0, 1-9, 10-19, 20-27",96.350365,76.923077,96.076642,73.846154,True,4


## locations + mobility

In [11]:
df_loc_mob = pd.read_csv("partition_results_mobility.csv")
df_loc_mob

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0
1,"0, 1-9, 10-19, 20-27",96.653144,65.714286,96.653144,66.857143,False,0
2,"0-4, 5-9, 10-14, 15-27",96.044625,56.000000,96.044625,55.428571,False,0
3,base,94.929006,48.000000,94.929006,48.000000,False,0
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",95.436105,44.571429,95.436105,44.000000,False,0
5,"0-4, 5-14, 15-27",96.800826,57.291667,96.800826,56.770833,True,0
6,"0, 1-9, 10-19, 20-27",96.388029,45.833333,96.388029,44.270833,True,0
7,"0-4, 5-9, 10-14, 15-27",95.046440,36.458333,95.046440,32.812500,True,0
8,base,94.427245,29.166667,94.427245,26.562500,True,0
9,"0, 1-4, 5-9, 10-14, 15-19, 20-27",93.911249,34.895833,93.911249,34.375000,True,0


In [12]:
df_loc_mob[df_loc_mob["train"] == df_loc_mob["train"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0


In [13]:
df_loc_mob[df_loc_mob["train (fs)"] == df_loc_mob["train (fs)"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0


In [14]:
df_loc_mob[df_loc_mob["test"] == df_loc_mob["test"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0


In [15]:
df_loc_mob[df_loc_mob["test (fs)"] == df_loc_mob["test (fs)"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
26,"0, 1-9, 10-19, 20-27",96.532847,66.153846,96.532847,69.230769,True,4


## locations + mobility + phone

In [16]:
df_loc_mob_phone = pd.read_csv("partition_results_mobility_phone.csv")
df_loc_mob_phone

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0
1,"0, 1-9, 10-19, 20-27",96.754564,68.571429,96.653144,65.714286,False,0
2,"0-4, 5-9, 10-14, 15-27",96.146045,58.285714,96.044625,55.428571,False,0
3,base,95.131846,47.428571,94.929006,48.000000,False,0
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",95.638945,42.285714,95.436105,44.000000,False,0
5,"0-4, 5-14, 15-27",96.904025,52.083333,96.800826,56.250000,True,0
6,"0, 1-9, 10-19, 20-27",96.594427,45.833333,96.388029,47.916667,True,0
7,"0-4, 5-9, 10-14, 15-27",95.149639,32.291667,95.046440,32.812500,True,0
8,base,94.633643,26.041667,94.427245,32.812500,True,0
9,"0, 1-4, 5-9, 10-14, 15-19, 20-27",94.220846,31.250000,93.911249,34.375000,True,0


In [18]:
df_loc_mob_phone[df_loc_mob_phone["train"] == df_loc_mob_phone["train"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0


In [19]:
df_loc_mob_phone[df_loc_mob_phone["train (fs)"] == df_loc_mob_phone["train (fs)"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0


In [20]:
df_loc_mob_phone[df_loc_mob_phone["test"] == df_loc_mob_phone["test"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
1,"0, 1-9, 10-19, 20-27",96.754564,68.571429,96.653144,65.714286,False,0


In [21]:
df_loc_mob_phone[df_loc_mob_phone["test (fs)"] == df_loc_mob_phone["test (fs)"].max()]

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0


## Comparison

In [22]:
pd.concat([
    df[df["train"] == df["train"].max()], 
    df_loc_mob[df_loc_mob["train"] == df_loc_mob["train"].max()], 
    df_loc_mob_phone[df_loc_mob_phone["train"] == df_loc_mob_phone["train"].max()]], axis=0)

,partition,train,test,train (fs),test (fs),SPLIT,TTF
25,"0-4, 5-14, 15-27",96.897810,61.538462,96.624088,64.615385,True,4
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0


In [23]:
pd.concat([
    df[df["train (fs)"] == df["train (fs)"].max()],
    df_loc_mob[df_loc_mob["train (fs)"] == df_loc_mob["train (fs)"].max()],
    df_loc_mob_phone[df_loc_mob_phone["train (fs)"] == df_loc_mob_phone["train (fs)"].max()]], axis=0)

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",96.855984,70.857143,96.653144,71.428571,False,0
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0


In [24]:
pd.concat([
    df[df["test"] == df["test"].max()],
    df_loc_mob[df_loc_mob["test"] == df_loc_mob["test"].max()],
    df_loc_mob_phone[df_loc_mob_phone["test"] == df_loc_mob_phone["test"].max()]], axis=0)

,partition,train,test,train (fs),test (fs),SPLIT,TTF
26,"0, 1-9, 10-19, 20-27",96.350365,76.923077,96.076642,73.846154,True,4
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0
1,"0, 1-9, 10-19, 20-27",96.754564,68.571429,96.653144,65.714286,False,0


In [25]:
pd.concat([
    df[df["test (fs)"] == df["test (fs)"].max()],
    df_loc_mob[df_loc_mob["test (fs)"] == df_loc_mob["test (fs)"].max()],
    df_loc_mob_phone[df_loc_mob_phone["test (fs)"] == df_loc_mob_phone["test (fs)"].max()]], axis=0)

,partition,train,test,train (fs),test (fs),SPLIT,TTF
26,"0, 1-9, 10-19, 20-27",96.350365,76.923077,96.076642,73.846154,True,4
26,"0, 1-9, 10-19, 20-27",96.532847,66.153846,96.532847,69.230769,True,4
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0


# Add CV


Do everything with

-> Split by the participant, and take one row, cross-validation (10-n), dailies =-> TBD

In [26]:
# Check balanced accuracy instead of raw and plot distributions
# Maybe look at the distr of the RMSE for regression

# Also, maybe use the std. instead of just the mean for features
# Try to use significant cross-validation
# Maybe apply feature selection:
#    * Automatic feature selection
#    * Manual: e.g. greatly reduce the number of location categories except for work and home
# Try to see the effect of some hyperparameters of the random forest

# Use only a few initial weeks in training for some patients
# Maybe try to improve the context with previous information

# Try different targets:
#    * The original sum_phq9 (not the column sum of 9 questions)
#    * Target each question separately
from collections import defaultdict
import cv
from data_processing import *
import data_processing as dp
import metrics

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


def train_cv(
    TYPE="classification",
    TARGET="value",
    SPLIT_BY_PARTICIPANT=False,
    TEST_TAKE_FIRST=0,
    SEED=550,
    N_SPLIT=5,
    return_csv=False,
    use_counts=False,
    feature_selection=False,
    partition=None,
    verbose=True,
    dailies_names=[]
):
    phq9 = dp.load_phq9_targets(DATADIR / "df_phq9.csv", type=TYPE, target=TARGET, partition=partition)
    locations = load_locations(DATADIR / "df_location_ratio.csv")
    demographics = load_demographics(DATADIR / "df_demographics.csv")
    
    dailies = [("locations", locations)]

    if "mobility" in dailies_names:
        mobility = load_passive_mobility(DATADIR / "df_passive_mobility_features.csv")
        dailies.append(("mobility", mobility))
    if "phone" in dailies_names:
        phone = load_passive_phone(
            DATADIR / "df_passive_phone_communication_features_brighten_v2.csv"
        )
        dailies.append(("phone", phone))


    combined, merge_result = dp.combine(
        phq9,
        dailies=dailies,
        constants=[demographics],
        prev_phq9=False,
    )
    combined = dp.rf_preprocess(combined)
    if return_csv:
        return combined

    # print('Train set shape:', x_train.shape)
    # print('Test set shape:', x_test.shape)
    # n = len(y_train) + len(y_test)
    # train_pct = len(y_train) / n * 100
    # test_pct = len(y_test) / n * 100
    # print(f'Ratio: {train_pct:.2f}%/{test_pct:.2f}%')
    # print()

    metric_dict = defaultdict(list)
    gen = (
        cv.per_patient_cv(combined, N_SPLIT, TEST_TAKE_FIRST)
        if SPLIT_BY_PARTICIPANT
        else cv.per_row_cv(combined, N_SPLIT)
    )
    for i, (x_train, x_test, y_train, y_test) in enumerate(gen):
        print(f"--- Split {i} ---")
        if x_train is None:
            print("Skipped, because take first emptied the test set.")
            continue
        if TYPE == "regression":

            def rmse(x, y):
                return np.sqrt(((x - y) ** 2).mean())

            model = RandomForestRegressor(
                n_estimators=300, n_jobs=-1, random_state=SEED
            )
            model.fit(x_train, y_train)

            train_rmse = rmse(y_train, model.predict(x_train))
            test_rmse = rmse(y_test, model.predict(x_test))
            print(f"Train set RMSE: {train_rmse:.4f}")
            print(f"Test set RMSE:  {test_rmse:.4f}")

            train_score = model.score(x_train, y_train)
            test_score = model.score(x_test, y_test)
            print(f"Train score:", train_score)
            print(f"Test score:", test_score)

            return train_score, test_score
        elif TYPE == "classification":

            model = RandomForestClassifier(
                n_estimators=300, n_jobs=-1, random_state=SEED
            )
            model.fit(x_train, y_train)

            y_full = np.concatenate((y_train, y_test))
            random_acc = (
                100 * (y_full == np.random.randint(0, 5, size=y_full.shape)).mean()
            )
            # train_acc = 100 * model.score(x_train, y_train)
            # test_acc = 100 * model.score(x_test, y_test)
            # print(f'Train set accuracy: {train_acc:.2f}%')
            # print(f'Test set accuracy:  {test_acc:.2f}%')
            print(f"Random full accuracy: {random_acc:.2f}%")

            # figs, axes = plt.subplots(1, 2, figsize=(16, 6))
            axes = [None, None]

            print("Train set:")
            train_bal, train_mean = metrics.accuracy_info(
                y_train,
                model.predict(x_train),
                prefix="Training",
                ax=axes[0],
                plot=False,
            )

            print("Test set:")
            test_bal, test_mean = metrics.accuracy_info(
                y_test, model.predict(x_test), prefix="Test", ax=axes[1], plot=False
            )

            metric_dict["train_bal"].append(train_bal)
            metric_dict["train_mean"].append(train_mean)
            metric_dict["test_bal"].append(test_bal)
            metric_dict["test_mean"].append(test_mean)

    for k, v in metric_dict.items():
        metric_dict[k] = np.array(v)

    return metric_dict

In [32]:
def all_combinations_scores_cv(dailies_names=[]):
    train_bal_scores = []
    train_mean_scores = []
    test_bal_scores = []
    test_mean_scores = []

    for partition in partitions:
        metric_dict = train_cv(
            TYPE="classification",
            TARGET="value",
            SPLIT_BY_PARTICIPANT=True,
            TEST_TAKE_FIRST=1,
            SEED=550,
            N_SPLIT=10,
            return_csv=False,
            use_counts=False,
            feature_selection=True,
            partition=partition,
            verbose=False,
            dailies_names=dailies_names,
        )
        train_bal_scores.append(metric_dict['train_bal'].mean())
        train_mean_scores.append(metric_dict['train_mean'].mean())
        test_bal_scores.append(metric_dict['test_bal'].mean())
        test_mean_scores.append(metric_dict['test_mean'].mean())

    df_cv = pd.DataFrame(
        {
            "partition": [
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
            ],
            "train bal": train_bal_scores,
            "train mean": train_mean_scores,
            "test bal": test_bal_scores,
            "test mean": test_mean_scores,
        }
    )
    dailies_substr = "_".join([str(x) for x in dailies_names])
    df_cv.to_csv("partition_results_" + dailies_substr + "_csv.csv", index=False)

    return df_cv

## locations (base)

In [33]:
df_loc_cv = all_combinations_scores_cv()
df_loc_cv

--- Split 0 ---
Random full accuracy: 22.22%
Train set:
Mean accuracy: 96.744%
Balanced accuracy: 95.323%
Test set:
Mean accuracy: 56.977%
Balanced accuracy: 43.385%
--- Split 1 ---
Random full accuracy: 19.72%
Train set:
Mean accuracy: 96.741%
Balanced accuracy: 95.437%
Test set:
Mean accuracy: 60.920%
Balanced accuracy: 47.381%
--- Split 2 ---
Random full accuracy: 18.60%
Train set:
Mean accuracy: 96.834%
Balanced accuracy: 95.370%
Test set:
Mean accuracy: 64.368%
Balanced accuracy: 48.033%
--- Split 3 ---
Random full accuracy: 19.55%
Train set:
Mean accuracy: 97.103%
Balanced accuracy: 95.817%
Test set:
Mean accuracy: 57.143%
Balanced accuracy: 46.525%
--- Split 4 ---
Random full accuracy: 19.29%
Train set:
Mean accuracy: 96.750%
Balanced accuracy: 96.075%
Test set:
Mean accuracy: 65.476%
Balanced accuracy: 54.244%
--- Split 5 ---
Random full accuracy: 19.81%
Train set:
Mean accuracy: 97.023%
Balanced accuracy: 95.524%
Test set:
Mean accuracy: 53.488%
Balanced accuracy: 38.943%
--- 

,partition,train bal,train mean,test bal,test mean
0,"0-4, 5-14, 15-27",0.958615,0.969631,0.484380,0.615711
1,"0, 1-9, 10-19, 20-27",0.965982,0.964601,0.425900,0.571642
2,"0-4, 5-9, 10-14, 15-27",0.953901,0.952022,0.391038,0.432041
3,base,0.950000,0.946058,0.386857,0.393573
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",0.954212,0.942799,0.305521,0.366428


## locations + mobility

In [34]:
df_loc_mob_cv = all_combinations_scores_cv(dailies_names=["mobility"])
df_loc_mob_cv

--- Split 0 ---
Random full accuracy: 18.95%
Train set:
Mean accuracy: 96.837%
Balanced accuracy: 95.379%
Test set:
Mean accuracy: 59.302%
Balanced accuracy: 43.519%
--- Split 1 ---
Random full accuracy: 21.71%
Train set:
Mean accuracy: 96.834%
Balanced accuracy: 96.125%
Test set:
Mean accuracy: 58.621%
Balanced accuracy: 40.286%
--- Split 2 ---
Random full accuracy: 20.24%
Train set:
Mean accuracy: 96.927%
Balanced accuracy: 95.388%
Test set:
Mean accuracy: 57.471%
Balanced accuracy: 38.561%
--- Split 3 ---
Random full accuracy: 20.41%
Train set:
Mean accuracy: 97.196%
Balanced accuracy: 95.911%
Test set:
Mean accuracy: 54.945%
Balanced accuracy: 34.059%
--- Split 4 ---
Random full accuracy: 22.74%
Train set:
Mean accuracy: 96.843%
Balanced accuracy: 96.169%
Test set:
Mean accuracy: 60.714%
Balanced accuracy: 47.145%
--- Split 5 ---
Random full accuracy: 19.64%
Train set:
Mean accuracy: 97.116%
Balanced accuracy: 95.579%
Test set:
Mean accuracy: 50.000%
Balanced accuracy: 35.749%
--- 

,partition,train bal,train mean,test bal,test mean
0,"0-4, 5-14, 15-27",0.959956,0.970563,0.398794,0.570182
1,"0, 1-9, 10-19, 20-27",0.968296,0.966371,0.350299,0.520229
2,"0-4, 5-9, 10-14, 15-27",0.955558,0.953792,0.369330,0.425992
3,base,0.953592,0.949505,0.301617,0.363834
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",0.955923,0.944569,0.253570,0.357235


## locations + mobility + phone

In [35]:
df_loc_mob_phone_cv = all_combinations_scores_cv(dailies_names=["mobility", "phone"])
df_loc_mob_phone_cv

--- Split 0 ---
Random full accuracy: 19.38%
Train set:
Mean accuracy: 96.930%
Balanced accuracy: 95.473%
Test set:
Mean accuracy: 58.140%
Balanced accuracy: 42.824%
--- Split 1 ---
Random full accuracy: 21.71%
Train set:
Mean accuracy: 96.927%
Balanced accuracy: 95.587%
Test set:
Mean accuracy: 57.471%
Balanced accuracy: 39.095%
--- Split 2 ---
Random full accuracy: 21.27%
Train set:
Mean accuracy: 97.020%
Balanced accuracy: 95.520%
Test set:
Mean accuracy: 59.770%
Balanced accuracy: 40.476%
--- Split 3 ---
Random full accuracy: 22.74%
Train set:
Mean accuracy: 97.290%
Balanced accuracy: 96.004%
Test set:
Mean accuracy: 54.945%
Balanced accuracy: 34.059%
--- Split 4 ---
Random full accuracy: 17.92%
Train set:
Mean accuracy: 96.936%
Balanced accuracy: 96.225%
Test set:
Mean accuracy: 55.952%
Balanced accuracy: 44.367%
--- Split 5 ---
Random full accuracy: 20.76%
Train set:
Mean accuracy: 97.209%
Balanced accuracy: 95.673%
Test set:
Mean accuracy: 48.837%
Balanced accuracy: 34.004%
--- 

,partition,train bal,train mean,test bal,test mean
0,"0-4, 5-14, 15-27",0.960056,0.971401,0.405143,0.573165
1,"0, 1-9, 10-19, 20-27",0.971236,0.968048,0.342328,0.507340
2,"0-4, 5-9, 10-14, 15-27",0.956129,0.954631,0.363029,0.417607
3,base,0.956474,0.951182,0.301767,0.364151
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",0.958524,0.947084,0.250794,0.348143
